# Linux Control Groups Demo

This notebook explores cgroups on Linux. In walking through this notebook you will examine how cgroups are presented in the Linux filesystem, explore creating a new cgroup and assigning a PID to the group and finally examine how Docker translates many resource limits into cgroup based configuration. 

This notebook has been tested on an EC2 instance running Amazon Linux 2. 

During the public demonstration of this notebook at the AWS Sydney Summit 2019, the notebook was executed on a c4.xlarge instance. 

## Initial Exploration of cgroups

Each process running in Linux is a member of a single cgroup per heirarchy. Cgroups are made visible via the /proc and /sys filesystems.

Firstly, you can find the cgroup heiarchies listed under the /sys filesystem:

In [ ]:
ls /sys/fs/cgroup

Now, lets look at the cgroups our shell is currently in. First, get the PID:

In [ ]:
echo $$

Now, lets find which cgroups this PID is currently mapped to:

In [ ]:
cat /proc/$$/cgroup

From the above, you can see for some heirarchies, the process is assigned to the root of the heirarchy: for example freezer& cpuset. Note that in other heirarchies, for example blkio, the process is assigned to a sub-folder within the heirarchy: /user.slice . 

Within each cgroup, there are virtual files which are used by the resource controller to manage the processes allocated to the group. There are also reserved files, such as 'tasks' which are used to identify, and map, which PIDs are in this part of the cgroup heirarchy:

In [ ]:
ls /sys/fs/cgroup/pids

From here in the root of the pids cgroup, you can see both the virtual files used by the resource controller, as we ll as the folder containing the user.slice section of the heirarchy we noted earlier

The user.slice folder contains its own version of the policy files, as well as its own tasks file:

In [ ]:
ls /sys/fs/cgroup/pids/user.slice/

Lets examine the tasks file in the user.slice folder. This file contains all of the PIDs that are in this part of the cgroup heiarchy, including our shell!

In [ ]:
cat /sys/fs/cgroup/pids/user.slice/tasks | grep $$

### Creating a new cgroup heirarchy: Controlling CPU Mapping

In this section, we will create a new cgroup heirarchy within the cpuset cgroup. Within this, we will show how we can limit a process to only use specific CPUs in the host by configuring these limits on a cgroup and placing a process within this part of the heirarchy. 

First, lets look at the controls available in the cpuset cgroup:

In [ ]:
ls /sys/fs/cgroup/cpuset

There are a number of controls available to manage the allocation of CPUs and memory to processes controlled by the cgroup.

To create a new section of the heirarchy, we simply create a folder within the base cgroup folder:

In [ ]:
sudo mkdir /sys/fs/cgroup/cpuset/containers-demo

In [ ]:
ls /sys/fs/cgroup/cpuset/containers-demo

To allow us to tweak the the cpuset virtual files we need to define baseline limits on memory and CPU cores. The kernel will not allow us to add a process to the this cgroup until this is complete. 

This demo is running a 4 vCPU EC2 instance (c4.xlarge) so we will define all 4 vCPUs as accessible initially:

In [ ]:
echo 0 | sudo tee /sys/fs/cgroup/cpuset/containers-demo/cpuset.mems
echo 0-3 | sudo tee /sys/fs/cgroup/cpuset/containers-demo/cpuset.cpus

Next, lets move our shell to the new cgroup we've just created by adding the PID to the tasks file:

In [ ]:
echo $$ | sudo tee /sys/fs/cgroup/cpuset/containers-demo/tasks

Now lets check which PIDs are mapped to this section of the cgroup heirarchy:

In [ ]:
cat /sys/fs/cgroup/cpuset/containers-demo/tasks

two processes! Our shell, but also the cat command as the shell is forking the cat process and the child process inherits its cgroup mapping from its parent.

Lets explore using cgroups to limit access to CPU Cores. Our host is a c4.xlarge which has 4 vCPU.

If we run sysbench with 4 threads, lets see what result we get:

In [ ]:
/usr/bin/sysbench cpu --threads=4 run

Now, lets modify the cpuset.cpus setting to limit the process to 2 of the 4 vCPUs:

In [ ]:
echo 2-3 | sudo tee /sys/fs/cgroup/cpuset/containers-demo/cpuset.cpus

Now if we re-run the same test, we should see approximately half the CPU performance: 

In [ ]:
/usr/bin/sysbench cpu --threads=4 run

## Docker and cgroups

Now lets look at the same thing with docker

Before proceeding, reset the Jupyter Kernel (under Kernel->Restart) to spawn a new shell which is not mapped to the cgroup we just created. 

Run the following commands to validate the kernel has a new PID:

In [ ]:
echo $$

In [ ]:
cat /proc/$$/cgroup

For this demo, we will use the Amazon Linux 2 Docker image to explore how Docker leverages cgroups. First, pull the image on to the host:

In [ ]:
docker pull amazonlinux:2

Now lets run the container in an infinite loop and tell Docker to set cpuset-cpus to 2-3, just like we did in our previous demo:

In [ ]:
docker run --cpuset-cpus 2-3 --rm -d --cidfile /tmp/docker_amazonlinux.cid amazonlinux:2 /bin/tail -f /dev/null
CONTAINER_CID=`cat /tmp/docker_amazonlinux.cid`

In [ ]:
docker ps

Docker creates a cgroup heirarchy for itself, and then within that a sub-heirarchy for each container based on the container id. In the above Docker command, we exported the container ID and we reference it here in the shell variable $CONTAINER_CID for briefness:

In [ ]:
ls /sys/fs/cgroup/cpuset/docker/$CONTAINER_CID/

If we inspect the cpuset.cpus value, we will see it matches that passed to the Docker command when we launched the container: 

In [ ]:
cat /sys/fs/cgroup/cpuset/docker/$CONTAINER_CID/cpuset.cpus

## Lab Clean Up

The following commands remove the cgroups created in this notebook. 

Prior to running the below commands, Click on Kernel->Restart in Jupyter to start a new underlying shell, as you will not be able to remove a cgroup if there is a process (ie the original Jupyter Kernel) still mapped to it. 

After restarting the kernel, execute the next line to remove the containers-demo cgroup:

In [ ]:
sudo rmdir /sys/fs/cgroup/cpuset/containers-demo

Execute the following command to remove the temporary container id file created by the Docker example in this lab:

In [ ]:
rm /tmp/docker_amazonlinux.cid